In [ ]:
#importing modules for web-scraping
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager 
import pandas as pd
import time
import math

#importing modules for the API key 
#requests are already imported 
import json

# Google API Key
from config import gkey


In [ ]:
#open the Chrome Driver web browser in order to do web-scraping 
executable_path = {'executable_path':'/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
#Web Scraping 1- need to interact with the job-seeking website first of all


jobs_find_url ="https://www.seek.com.au/jobs/in-All-Australia?keywords=%22data%20%20analyst%22"
browser.visit(jobs_find_url)
jobs_html=browser.html 

The first step is to get the web-page url from the seek jobs website where the job includes "data analyst". The job search for this page is limited to all of Australia


Next, the number of pages that the search included needed to be obtained so that we know how many pages to loop through. It was found that there were 22 listings per page, and thus the numebr of jobs was divided by 22. After this it was noticed that the job advertisements just duplicate. Thus a decision was made to round up the number of pages/22 and then delete duplicate entries later. 

In [ ]:
# Retrieve page with the requests module

#it was noticed that the seek page had 22 jobs listed per page. Thus in order to obtain the number of pages, first need to obtain the number of pages:
#Number of jobs listed at the top of the page, the following code obtains this:
job_soup=bs(jobs_html, 'html.parser')
job_number=job_soup.find('span', class_="_3FrNV7v _3PZrylH _2heRYaN").text

number=job_number.strip('jobs found')

number_1=float(number)
print(number_1)

number_pages=math.ceil(number_1/22)
print(number_pages)

time.sleep(2)


The code in the excerpt below saves the job title, location and company for data analyst jobs throughout Australia into lists. This was done through using looping and webscraping. 

In [5]:
#getting the browser html
text_list=[]
job_location=[]
job_title1=[]
job_company=[]
x=range(1,number_pages+1)
for n in x:
    seek_url=f"https://www.seek.com.au/jobs/in-All-Australia?keywords=%22data%20%20analyst%22&page={n}"
    browser.visit(seek_url)
    seek_html=browser.html  
    seek_soup=bs(seek_html, 'html.parser')
    all_text=seek_soup.find('div', class_='_3MPUOLE')
    job_title=all_text.find_all('span', class_='_3FrNV7v _2IOW3OW HfVIlOd _2heRYaN E6m4BZb')
    extra_class=all_text.find_all('span', class_="Eadjc1o")
    job_characteristics=all_text.find_all('span', class_="_3FrNV7v _3PZrylH E6m4BZb")

    #converting each of the elements to text 
    
    #modifying the job location element and tidying up this element
    for element in extra_class:
        element1=element.text
        if "location:" in element1:
            element2=element1.lstrip('location: ')
            job_location.append(element2) 
    
    job_title=all_text.find_all('span', class_='_3FrNV7v _2IOW3OW HfVIlOd _2heRYaN E6m4BZb') 
    
    #editing both the job title and job characterstics
    for item in job_title:
        job1=item.text
        job_title1.append(job1)
    
    for company in job_characteristics:
        company1=company.text
        if company1[:3]=='at ':
            company2=company1.lstrip('at ')
            job_company.append(company2) 
    
    time.sleep(3)
            

In [6]:
browser.quit()

The lists were then combined in a pandas dataframe

In [7]:
#zip two lists together 

Job_df = pd.DataFrame(
    {'Job Title': job_title1,
     'Job Location': job_location,
     'Job Company': job_company,
    })


In [8]:
Job_df.head(20)

,Job Title,Job Location,Job Company
0,Financial and Business Data Analyst,Adelaide,Brentnalls SA
1,Graduate Data Analyst & Programmer,Gold Coast,Tebbutt Research
2,Data Analyst/Manager,Sydney,Finite IT Recruitment Solutions
3,Data Analyst,Sydney,Naviro
4,Data Analyst,Sydney,Quay Appointments
5,Senior Data Analyst,Sydney,Ethos BeathChapman
6,Data Analyst,Melbourne,Department of Justice and Community Safety
7,Data Analyst,Melbourne,Department of Justice and Community Safety
8,Senior Data Analyst,ACT,Talent Street
9,Scientist (Data Analyst),Southern Highlands & Tablelands,"Department of Planning, Industry and Environment"


Duplicates were then removed from the dataset

In [18]:
#removing duplicates if all three columns are the same: 

Job_df2=Job_df.drop_duplicates()

In order to use the Leaflet library, the longitude and latitude co-ordinates needed to be obtained for each job, This will be done using the Google Places API. First, a search string column was added to the dataframe. This Search Engins column will be used as a parameter in the Google Places API. 

In [19]:
#setting up search to use in the API call 
Job_df2["Search Engine"] = Job_df2["Job Company"].astype(str) + " " + Job_df2["Job Location"].astype(str)

/Users/dhanyamaheswaran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
Job_df2.head()

,Job Title,Job Location,Job Company,Search Engine
0,Financial and Business Data Analyst,Adelaide,Brentnalls SA,Brentnalls SA Adelaide
1,Graduate Data Analyst & Programmer,Gold Coast,Tebbutt Research,Tebbutt Research Gold Coast
2,Data Analyst/Manager,Sydney,Finite IT Recruitment Solutions,Finite IT Recruitment Solutions Sydney
3,Data Analyst,Sydney,Naviro,Naviro Sydney
4,Data Analyst,Sydney,Quay Appointments,Quay Appointments Sydney


The below code adds Lat and Lng columns to the dataframe to set up the dataframe further for the Google API 

In [21]:
#adding latitude and longitude columns
Job_df2['Lat'] = ""
Job_df2['Lng']=""

/Users/dhanyamaheswaran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/dhanyamaheswaran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


The following code loops through all the search engine column and outputs a Lat and Lng from the Google Places API. Try and Except is used so that if there is an error, the code will not automatically stop. 

In [22]:
for index, row in Job_df2.iterrows():
    
    search_string=row["Search Engine"]
    
    try:
        url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={search_string}&inputtype=textquery&fields=photos,formatted_address,name,rating,opening_hours,geometry&key={gkey}'
        # Run request
        response = requests.get(url)
        search_geo = response.json()
        
    
        Job_df2.loc[index, 'Lat'] = search_geo["candidates"][0]['geometry']['location']['lat']
        Job_df2.loc[index, "Lng"]=search_geo["candidates"][0]['geometry']['location']['lng']

    
    #in the case of a JSONDecode Error, this ensures the code will keep running
    except (KeyError, IndexError, ValueError):
        Job_df2.loc[index, 'Lat'] = "No Coordinates"
        Job_df2.loc[index, 'Lng'] = "No Coordinates"


/Users/dhanyamaheswaran/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/dhanyamaheswaran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/Users/dhanyamaheswaran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [23]:
Job_df2.head(15)

,Job Title,Job Location,Job Company,Search Engine,Lat,Lng
0,Financial and Business Data Analyst,Adelaide,Brentnalls SA,Brentnalls SA Adelaide,-34.9036,138.571
1,Graduate Data Analyst & Programmer,Gold Coast,Tebbutt Research,Tebbutt Research Gold Coast,-27.9706,153.413
2,Data Analyst/Manager,Sydney,Finite IT Recruitment Solutions,Finite IT Recruitment Solutions Sydney,-33.8667,151.208
3,Data Analyst,Sydney,Naviro,Naviro Sydney,-33.8635,151.208
4,Data Analyst,Sydney,Quay Appointments,Quay Appointments Sydney,-33.864,151.206
5,Senior Data Analyst,Sydney,Ethos BeathChapman,Ethos BeathChapman Sydney,-33.864,151.207
6,Data Analyst,Melbourne,Department of Justice and Community Safety,Department of Justice and Community Safety Mel...,-37.8127,144.97
8,Senior Data Analyst,ACT,Talent Street,Talent Street ACT,-35.3205,149.097
9,Scientist (Data Analyst),Southern Highlands & Tablelands,"Department of Planning, Industry and Environment","Department of Planning, Industry and Environme...",-34.4249,150.899
11,APS6 Policy/Data Analyst,ACT,First People Recruitment Solutions Pty Ltd,First People Recruitment Solutions Pty Ltd ACT,-35.2787,149.129


In order to double check, the entries where lat and lng coordinates were not found were just displayed. 

In [24]:
#check the ones that have no co-ordinates
Job_df2.loc[Job_df2['Lat'] == "No Coordinates"]

,Job Title,Job Location,Job Company,Search Engine,Lat,Lng
26,Senior Data Analyst,ACT,Adaps IT,Adaps IT ACT,No Coordinates,No Coordinates
89,Business Data Manager,Sydney,FEfundinfo Australia,FEfundinfo Australia Sydney,No Coordinates,No Coordinates
108,Data Analyst,"Newcastle, Maitland & Hunter",Karlka Recruiting Group,"Karlka Recruiting Group Newcastle, Maitland & ...",No Coordinates,No Coordinates
125,Data Analyst,Sydney,ABC,ABC Sydney,No Coordinates,No Coordinates
131,Data Analyst,Sunshine Coast,NOVA Partners,NOVA Partners Sunshine Coast,No Coordinates,No Coordinates
132,Experience Report Developer,Darwin,XIPNOS Pty Ltd,XIPNOS Pty Ltd Darwin,No Coordinates,No Coordinates
146,Data Analyst,ACT,First Grade Recruitment,First Grade Recruitment ACT,No Coordinates,No Coordinates
153,Data Analyst,Sydney,The Retail Score,The Retail Score Sydney,No Coordinates,No Coordinates
169,Warehouse Data Analyst,Melbourne,Caterpillar of Australia,Caterpillar of Australia Melbourne,No Coordinates,No Coordinates
170,COVID-19 Vaccination Program Data Analyst - Ex...,Sydney,Health System Support Group,Health System Support Group Sydney,No Coordinates,No Coordinates


In order to data-clean, all Jobs where no coordinates were found was then deleted from the dataframe and it was exported as a csv

In [25]:
#Then delete the ones with no co-ordinates
final_Job_df=Job_df2.loc[Job_df2['Lat'] != "No Coordinates"]

In [26]:
#Finally export it to a csv
final_Job_df.to_csv("Final_job_df.csv")